In [1]:
import netket as nk
import json
from qutip import *
import numpy as np
import time
import multiprocessing as mp
from collections import OrderedDict
from pickle import dump
import os
import matplotlib.pyplot as plt
import scipy
from matplotlib import gridspec
from functools import reduce
plt.style.use('seaborn')
from scipy.stats import norm
import sys
sys.path.append("/home/victor1/penalty_method")
import expect_grad_ex
import vmc_ex
import jax
import pickle
import copy
import optax

In [2]:
def CSHam(N, B, Ak):
    # Make graph with of length N with no periodic boundary conditions
    g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
    # Spin based Hilbert Space
    hilbertSpace = nk.hilbert.Spin(s=0.5, N=g.n_nodes)
    # Define spin operators with \hbar set to 1
    sz = 0.5 * np.array([[1, 0], [0, -1]])
    sx = 0.5 * np.array([[0, 1], [1, 0]])
    sy = 0.5 * np.array([[0, -1j], [1j, 0]])
    operators = []
    sites = []
    # Central spin term
    operators.append((B * sz).tolist()) #array to list(ordered and changeable)
    sites.append([0])
    # Interaction term
    itOp = np.kron(sz, sz) + np.kron(sx, sx) + np.kron(sy, sy) #kronecker product here
    for i in range(N - 1):
        operators.append((Ak[i] * itOp).tolist())
        sites.append([0, (i+1)])  #pretty convoluted indexing, but ok
    # Create hamiltonian
    hamiltonian = nk.operator.LocalOperator(hilbertSpace, operators=operators, acting_on=sites, dtype=complex)
    #acting_on specifier necessary as this is a central spin model
    return hamiltonian, hilbertSpace

def exactDiagonalization_full(hamiltonian):
    # Changes Hamiltonian to matrix form
    haMatrix = hamiltonian.to_dense()
    # Gets eigenvalues and vectors
    eigenValues, v = np.linalg.eigh(haMatrix)
    # Orders from smallest to largest
    eigenVectors = [v[:, i] for i in range(len(eigenValues))]
    return eigenValues, eigenVectors

# Error Calculation (Input: the found state, the state from exact diagonalization, the found energy, the energy from exact diagonalization)
def err_sta(state, edState,N):
    overlap = np.dot(state.conj().reshape(2**N, 1).T, edState.reshape(2**N, 1))
    waveFunctionErr = 1 - (np.linalg.norm(overlap))**2
    return waveFunctionErr

# NetKet RBM with stochastic reconfiguration descent
class RBM:
    def __init__(self, N, hamiltonian, hilbertSpace, machine):
        # Assign inputsv[:, i]
        self.hamiltonian, self.hilbertSpace, self.machine, self.N = hamiltonian, hilbertSpace, machine, N
        # Define sampler
        self.sampler = nk.sampler.MetropolisLocal(hilbert=hilbertSpace)
        # Define optimizer
        self.optimizer = nk.optimizer.Sgd(learning_rate=0.005)
        # Define Stochastic reconfiguration
        self.sr = nk.optimizer.SR(diag_shift=0.01) #diagnol shift, its role as regularizer? seems to take a different form as 
        #compared to the version I have seen
        # Variational state
        self.vs = nk.vqs.MCState(self.sampler, self.machine, n_samples=1000, n_discard_per_chain=100) #discarded number of samples 
        #at the beginning of the MC chain

    # Output is the name of the output file in which the descent data is stored
    def __call__(self, output, state_list, shift_list):
        self.vs.init_parameters(jax.nn.initializers.normal(stddev=0.25))
        gs = vmc_ex.VMC_ex(hamiltonian=self.hamiltonian, optimizer=self.optimizer, variational_state=self.vs, preconditioner=self.sr, 
                   state_list = state_list, shift_list = shift_list)
        # Start timing
        start = time.time()
        # Set the output files as well as number of iterations in the descent
        gs.run(out=output, n_iter=300,show_progress = False)
        end = time.time()
        runTime = end - start
        # Import the data from log file
        data = json.load(open(output + '.log'))
        # Extract the relevant information
        # iters = data["Energy"]["iters"]
        energy_RBM = data["Energy"]["Mean"]["real"] #get the real part of the mean energy
       
        # finalEng = energy_RBM[-1]
        finalEng = reduce(lambda x, y: x if y is None else y, energy_RBM)
        # Get machine statethe state of the machine as an array
        state = self.vs.to_array()
        # Outputs the final energy, the final state, and the runtime
        parameters = self.vs.parameters
        # Outputs the final energy, the final state, and the runtime
        return finalEng, state, parameters


In [3]:
N = 6
Ak = []

alpha = 1   #density of RBM
M = alpha*N
# Constant A
B = 0.95
# Variable A
A = N/2
N0 = N/2
for i in range(N-1):
    # Constant A
    #Ak_i = 1
    # Variable A
    Ak_i = A / (N0) * np.exp(-i / N0)
    Ak.append(Ak_i)
    
# Define hamiltonian and hilbert space
ha, hi = CSHam(N,B,Ak)

#Exact Diagonalization
e, v = exactDiagonalization_full(ha)

In [8]:
ma = nk.models.RBM(alpha=1, dtype=complex,use_visible_bias=True, use_hidden_bias=True)
sampler = nk.sampler.MetropolisLocal(hilbert=hi)
vs1 = nk.vqs.MCState(sampler, ma, n_samples=1000, n_discard_per_chain=100)
vs1.init_parameters(jax.nn.initializers.normal(stddev=0.25))

/home/victor1/ENV/lib/python3.8/site-packages/netket/vqs/mc/mc_state/state.py:58: UserWarning: n_samples=1000 (1000 per MPI rank) does not divide n_chains=16, increased to 1008 (1008 per MPI rank)
  warnings.warn(


In [9]:
import pickle
fileName = "2021_summer_data/2022_winter_data/post_selection_data/parameter_N6i1j41_best.json"
file = open(fileName,'rb')
a = pickle.load(file)
vs1.parameters = a

In [12]:
gs0 = vs1.to_array()
overlap0 = np.dot(gs0.conj().reshape(2**N, 1).T, v[1].reshape(2**N, 1))
print(1-np.linalg.norm(overlap0)**2)

0.00022542138505921017


In [17]:
vs1.n_samples = 10000
vs1.expect(ha)

-1.27651-0.00012j ± 0.00069 [σ²=0.00393, R̂=1.0001]

In [10]:
print(e)

[-1.41524943 -1.32771232 -1.27145158 -1.21988005 -1.19876478 -1.11306153
 -1.11089347 -1.0858455  -1.07493451 -0.98756256 -0.93938085 -0.9338297
 -0.91135368 -0.90173021 -0.8935346  -0.8730331  -0.80315108 -0.69343838
 -0.62406881 -0.53438139 -0.50864167 -0.38721415 -0.36979703 -0.31053015
 -0.25413804 -0.18236887 -0.17478303 -0.09929506 -0.04515056  0.04131016
  0.10087815  0.24035625  0.35185493  0.41431648  0.41522106  0.43187103
  0.44636995  0.45165595  0.4549342   0.46446678  0.47988562  0.51981148
  0.5259888   0.53022274  0.53638579  0.59887303  0.65252521  0.6527589
  0.69325189  0.71139475  0.71240188  0.73158247  0.74761339  0.75153445
  0.78431651  0.86470533  0.8726142   0.88383632  0.88996922  0.97406376
  0.99132177  1.03767668  1.09885071  1.19035625]


In [13]:
b = [1,2,3]
print(len(b))

3


In [17]:
print(v[1])

[ 0.00000000e+00+0.j  8.68074001e-18+0.j  6.99963545e-17+0.j
  8.51002813e-02+0.j -3.03045015e-17+0.j  1.06772713e-01+0.j
  7.20250825e-02+0.j -1.11022302e-16+0.j  5.55111512e-17+0.j
  1.45728807e-01+0.j  9.02916327e-02+0.j -2.77555756e-17+0.j
  8.20021891e-02+0.j -1.21430643e-17+0.j -1.73472348e-17+0.j
 -4.33680869e-19+0.j -1.30673467e-16+0.j  2.09694012e-01+0.j
  1.22876807e-01+0.j -2.44000645e-17+0.j  1.04944356e-01+0.j
 -2.00371161e-17+0.j -1.58538582e-17+0.j  2.89581880e-21+0.j
  1.05609562e-01+0.j -1.28237254e-17+0.j -1.13215280e-17+0.j
 -2.43897233e-20+0.j -8.60367791e-18+0.j -2.51747361e-20+0.j
 -2.73011577e-20+0.j  0.00000000e+00+0.j  3.90806640e-17+0.j
 -7.43510967e-01+0.j -3.84357186e-01+0.j  7.24072751e-17+0.j
 -2.77039610e-01+0.j  5.32716949e-17+0.j  3.87480956e-17+0.j
  5.99453854e-20+0.j -2.21278660e-01+0.j  2.32066047e-17+0.j
  2.07938251e-17+0.j  4.67288877e-20+0.j  1.19013474e-17+0.j
  2.20053374e-20+0.j -2.06112242e-20+0.j  0.00000000e+00+0.j
 -1.80457511e-01+0.j  1.